In [1]:
import requests
import lxml
from pprint import pprint
from bs4 import BeautifulSoup
from selenium import webdriver


url = "https://stats.nba.com/player/1628369/boxscores-traditional/"

# page = requests.get(url,'lxml')

# soup = BeautifulSoup(page.content)
# pprint(soup.prettify()[:1000])


In [2]:
driver = webdriver.Chrome()
driver.get(url)

In [3]:
soup = BeautifulSoup(driver.page_source)
len(soup.find_all('tr'))

58

In [4]:
row_tags = soup.find_all('tr')
row_tags[0].text

'\nMatch Up\nW/L\nMIN\nPTS\nFGM\nFGA\nFG%\n3PM\n3PA\n3P%\nFTM\nFTA\nFT%\nOREB\nDREB\nREB\nAST\nSTL\nBLK\nTOV\nPF\n+/-\n'

In [5]:
row_tags[1].text

'\n\nDec 25, 2019 - BOS @ TOR\n\nW\n35\n11\n\n            5\n\n\n            18\n\n27.8\n\n            1\n\n\n            3\n\n33.3\n0\n0\n0.0\n\n            4\n\n\n            5\n\n\n            9\n\n\n            6\n\n\n            2\n\n\n            0\n\n\n            3\n\n4\n21\n'

In [6]:
headers = row_tags[0].text.replace('\n',',')
if headers[0]==',':
    headers = headers[1:]
headers

'Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,'

In [7]:
headers = row_tags[0].text.split('\n')
row1 = row_tags[1].text.split('\n')
row2 = row_tags[2].text.split('\n')

print(len(headers),len(row1),len(row2))
print(headers,row1,row2,sep='\n\n')

24 48 48
['', 'Match Up', 'W/L', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-', '']

['', '', 'Dec 25, 2019 - BOS @ TOR', '', 'W', '35', '11', '', '            5', '', '', '            18', '', '27.8', '', '            1', '', '', '            3', '', '33.3', '0', '0', '0.0', '', '            4', '', '', '            5', '', '', '            9', '', '', '            6', '', '', '            2', '', '', '            0', '', '', '            3', '', '4', '21', '']

['', '', 'Dec 22, 2019 - BOS vs. CHA', '', 'W', '36', '39', '', '            15', '', '', '            29', '', '51.7', '', '            4', '', '', '            9', '', '44.4', '5', '5', '100', '', '            1', '', '', '            11', '', '', '            12', '', '', '            2', '', '', '            1', '', '', '            3', '', '', '            2', '', '1', '10', '']


In [8]:
row_data = row_tags[1].text.replace('\n',',')
row_data

',,Dec 25, 2019 - BOS @ TOR,,W,35,11,,            5,,,            18,,27.8,,            1,,,            3,,33.3,0,0,0.0,,            4,,,            5,,,            9,,,            6,,,            2,,,            0,,,            3,,4,21,'

### Summary:
- Using the .text and splitting may not be the best strategy
- Try searching for tags like `td` inside of the table rows for better extraction

In [9]:
head = row_tags[0]
print(len(head.find_all('th')))


column_names = [t.text for t in head.find_all('th')]
column_names

22


['Match Up',
 'W/L',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3PM',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 '+/-']

In [10]:
row = row_tags[1]
len(row.find_all('td'))
[print(i,':',t.text) for i,t in enumerate(row.find_all('td'))]

0 : 
Dec 25, 2019 - BOS @ TOR

1 : W
2 : 35
3 : 11
4 : 
            5

5 : 
            18

6 : 27.8
7 : 
            1

8 : 
            3

9 : 33.3
10 : 0
11 : 0
12 : 0.0
13 : 
            4

14 : 
            5

15 : 
            9

16 : 
            6

17 : 
            2

18 : 
            0

19 : 
            3

20 : 4
21 : 21


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [11]:
row_data = [t.text.strip('\n').strip() for t in row.find_all('td')]
len(row_data)

22

In [12]:
import pandas as pd
pd.set_option('display.max_columns',0)
## Get header/column names
head = row_tags[0]
column_names = [t.text for t in head.find_all('th')]

data = []
for row in row_tags[1:]:
    row_data = [t.text.strip('\n').strip() for t in row.find_all('td')]
    data.append(row_data)

df = pd.DataFrame(data,columns=column_names)
df.head(5)

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,"Dec 25, 2019 - BOS @ TOR",W,35,11,5,18,27.8,1,3,33.3,0,0,0.0,4,5,9,6,2,0,3,4,21
1,"Dec 22, 2019 - BOS vs. CHA",W,36,39,15,29,51.7,4,9,44.4,5,5,100,1,11,12,2,1,3,2,1,10
2,"Dec 20, 2019 - BOS vs. DET",W,27,26,10,17,58.8,2,7,28.6,4,4,100,0,5,5,3,2,0,0,1,10
3,"Dec 18, 2019 - BOS @ DAL",W,37,24,10,23,43.5,2,10,20.0,2,3,66.7,2,6,8,1,0,1,3,3,-2
4,"Dec 12, 2019 - BOS vs. PHI",L,34,15,5,18,27.8,3,8,37.5,2,2,100,1,2,3,4,0,1,1,4,-2


In [13]:
df.dtypes

Match Up    object
W/L         object
MIN         object
PTS         object
FGM         object
FGA         object
FG%         object
3PM         object
3PA         object
3P%         object
FTM         object
FTA         object
FT%         object
OREB        object
DREB        object
REB         object
AST         object
STL         object
BLK         object
TOV         object
PF          object
+/-         object
dtype: object

In [14]:

df.columns

Index(['Match Up', 'W/L', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA',
       '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', '+/-'],
      dtype='object')

In [15]:
float_cols= [col for col in df.columns if '%' in col]
float_cols

['FG%', '3P%', 'FT%']

In [16]:
obj_cols= ['Match Up','W/L']
int_cols = [col for col in df.columns if (col not in float_cols) and (col not in obj_cols)]
int_cols

['MIN',
 'PTS',
 'FGM',
 'FGA',
 '3PM',
 '3PA',
 'FTM',
 'FTA',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 '+/-']

In [17]:

df[~df.isna()]

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,"Dec 25, 2019 - BOS @ TOR",W,35,11,5,18,27.8,1,3,33.3,0,0,0.0,4,5,9,6,2,0,3,4,21
1,"Dec 22, 2019 - BOS vs. CHA",W,36,39,15,29,51.7,4,9,44.4,5,5,100,1,11,12,2,1,3,2,1,10
2,"Dec 20, 2019 - BOS vs. DET",W,27,26,10,17,58.8,2,7,28.6,4,4,100,0,5,5,3,2,0,0,1,10
3,"Dec 18, 2019 - BOS @ DAL",W,37,24,10,23,43.5,2,10,20.0,2,3,66.7,2,6,8,1,0,1,3,3,-2
4,"Dec 12, 2019 - BOS vs. PHI",L,34,15,5,18,27.8,3,8,37.5,2,2,100,1,2,3,4,0,1,1,4,-2
5,"Dec 11, 2019 - BOS @ IND",L,37,16,4,13,30.8,2,8,25.0,6,6,100,0,6,6,3,2,0,3,2,-8
6,"Dec 09, 2019 - BOS vs. CLE",W,30,19,7,14,50.0,4,6,66.7,1,2,50.0,0,11,11,2,2,2,4,1,24
7,"Dec 06, 2019 - BOS vs. DEN",W,30,26,10,19,52.6,4,8,50.0,2,2,100,0,7,7,5,1,0,1,1,14
8,"Dec 04, 2019 - BOS vs. MIA",W,34,19,6,19,31.6,2,12,16.7,5,6,83.3,1,4,5,3,1,3,3,2,31
9,"Dec 01, 2019 - BOS @ NYK",W,37,30,10,23,43.5,5,11,45.5,5,6,83.3,0,6,6,7,2,2,1,4,11


In [18]:

df.dropna(inplace=True)

In [19]:
df[int_cols] = df[int_cols].astype('int')
df[float_cols] = df[float_cols].astype('float')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 22 columns):
Match Up    28 non-null object
W/L         28 non-null object
MIN         28 non-null int64
PTS         28 non-null int64
FGM         28 non-null int64
FGA         28 non-null int64
FG%         28 non-null float64
3PM         28 non-null int64
3PA         28 non-null int64
3P%         28 non-null float64
FTM         28 non-null int64
FTA         28 non-null int64
FT%         28 non-null float64
OREB        28 non-null int64
DREB        28 non-null int64
REB         28 non-null int64
AST         28 non-null int64
STL         28 non-null int64
BLK         28 non-null int64
TOV         28 non-null int64
PF          28 non-null int64
+/-         28 non-null int64
dtypes: float64(3), int64(17), object(2)
memory usage: 5.0+ KB


## Now Use Selenium to Select the Last 2 Seasons and repeat

In [20]:

sel_season = soup.find_all('select',attrs=dict(name='Season'))
sel_season[0].find_all('option')

[<option label="2019-20" selected="selected" value="object:212">2019-20</option>,
 <option label="2018-19" value="object:213">2018-19</option>,
 <option label="2017-18" value="object:214">2017-18</option>]

In [21]:
option = driver.find_element_by_css_selector('option')
option.click()

In [22]:
season = driver.find_element_by_xpath("/html/body/main/div[2]/div/div/div[3]/div/div/div/div[1]/div[1]/div/div/label/select")
season.click()



#### Page URLS May be easier to generate than click on interface


In [23]:
url_2019_20 = 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2019-20&SeasonType=Regular%20Season'
url_2018_19 = 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2018-19&SeasonType=Regular%20Season'
url_2017_18 = 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2017-18&SeasonType=Regular%20Season'

## SEPARATING ABOVE INTO F_STRING 
player_id = '1628369'
base_url = f"https://stats.nba.com/player/{player_id}/boxscores-traditional/"



seasons= {}
seasons[2019] = dict(start_YR = '2019', end_yr = '20')
seasons[2018] = dict(start_YR = '2018', end_yr = '19')
seasons[2017] = dict(start_YR = '2017', end_yr = '18')

## seasons have 4 digit start year and 2 digit end year
season_urls = []
for year in seasons:
    start_YR = seasons[year]['start_YR']
    end_yr = seasons[year]['end_yr']
    season_suffix = f"?Season={start_YR}-{end_yr}&SeasonType=Regular%20Season"
    full_url = base_url+season_suffix
    season_urls.append(full_url)

print(season_urls)



['https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2019-20&SeasonType=Regular%20Season', 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2018-19&SeasonType=Regular%20Season', 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2017-18&SeasonType=Regular%20Season']


In [72]:
# for url in season_urls:
#     driver.get(url)
# # driver.get(full_url)

In [24]:
import nba_stats as ns

df = ns.get_season_df(season_urls[0])

fsds_1007219  v0.5.13 loaded.  Read the docs: https://fsds.readthedocs.io/en/latest/ 
> For convenient loading of standard modules use: `>> from fsds_100719.imports import *`



In [29]:
df['date'] = df['Match Up'].apply(lambda x: x.split('-')[0])
df['teams'] = df['Match Up'].apply(lambda x: x.split('-')[1])

In [32]:
import pandas as pd
df['date'] = pd.to_datetime(df['date'])
df.set_index('date')

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,teams
date,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25,"Dec 25, 2019 - BOS @ TOR",W,35,11,5,18,27.8,1,3,33.3,0,0,0.0,4,5,9,6,2,0,3,4,21,BOS @ TOR
2019-12-22,"Dec 22, 2019 - BOS vs. CHA",W,36,39,15,29,51.7,4,9,44.4,5,5,100.0,1,11,12,2,1,3,2,1,10,BOS vs. CHA
2019-12-20,"Dec 20, 2019 - BOS vs. DET",W,27,26,10,17,58.8,2,7,28.6,4,4,100.0,0,5,5,3,2,0,0,1,10,BOS vs. DET
2019-12-18,"Dec 18, 2019 - BOS @ DAL",W,37,24,10,23,43.5,2,10,20.0,2,3,66.7,2,6,8,1,0,1,3,3,-2,BOS @ DAL
2019-12-12,"Dec 12, 2019 - BOS vs. PHI",L,34,15,5,18,27.8,3,8,37.5,2,2,100.0,1,2,3,4,0,1,1,4,-2,BOS vs. PHI
2019-12-11,"Dec 11, 2019 - BOS @ IND",L,37,16,4,13,30.8,2,8,25.0,6,6,100.0,0,6,6,3,2,0,3,2,-8,BOS @ IND
2019-12-09,"Dec 09, 2019 - BOS vs. CLE",W,30,19,7,14,50.0,4,6,66.7,1,2,50.0,0,11,11,2,2,2,4,1,24,BOS vs. CLE
2019-12-06,"Dec 06, 2019 - BOS vs. DEN",W,30,26,10,19,52.6,4,8,50.0,2,2,100.0,0,7,7,5,1,0,1,1,14,BOS vs. DEN
2019-12-04,"Dec 04, 2019 - BOS vs. MIA",W,34,19,6,19,31.6,2,12,16.7,5,6,83.3,1,4,5,3,1,3,3,2,31,BOS vs. MIA


In [33]:
import fsds_100719 as fs 
fs.reload(ns)

Reloading...



<module 'nba_stats' from '/Users/jamesirving/Documents/GitHub/_MY_SIDE_PROJECTS/nba_fantasy_fanduel/nba_stats.py'>

In [34]:
df2 = ns.get_season_df(season_urls[1])
df2

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,teams
date,,,,,,,,,,,,,,,,,,,,,,,
2019-04-07,"Apr 07, 2019 - BOS vs. ORL",L,6,0,0,2,0.0,0,0,0.0,0,0,0.0,0,1,1,0,0,0,0,0,1,BOS vs. ORL
2019-04-05,"Apr 05, 2019 - BOS @ IND",W,34,22,9,19,47.4,1,5,20.0,3,4,75.0,1,6,7,2,3,1,1,2,15,BOS @ IND
2019-04-03,"Apr 03, 2019 - BOS @ MIA",W,39,16,6,17,35.3,1,3,33.3,3,4,75.0,0,7,7,3,1,1,3,0,16,BOS @ MIA
2019-04-01,"Apr 01, 2019 - BOS vs. MIA",W,34,19,7,15,46.7,4,7,57.1,1,2,50.0,1,5,6,5,1,0,0,0,15,BOS vs. MIA
2019-03-30,"Mar 30, 2019 - BOS @ BKN",L,30,9,4,9,44.4,1,1,100.0,0,0,0.0,1,1,2,3,1,0,2,4,-12,BOS @ BKN
2019-03-29,"Mar 29, 2019 - BOS vs. IND",W,24,11,3,6,50.0,3,3,100.0,2,4,50.0,0,5,5,4,0,0,4,2,16,BOS vs. IND
2019-03-26,"Mar 26, 2019 - BOS @ CLE",W,34,21,9,17,52.9,3,5,60.0,0,0,0.0,2,3,5,4,1,0,1,4,17,BOS @ CLE
2019-03-23,"Mar 23, 2019 - BOS @ CHA",L,37,12,5,17,29.4,2,7,28.6,0,0,0.0,1,5,6,4,0,1,1,1,-19,BOS @ CHA
2019-03-20,"Mar 20, 2019 - BOS @ PHI",L,33,13,6,12,50.0,1,2,50.0,0,0,0.0,1,5,6,4,0,1,1,3,-17,BOS @ PHI


In [35]:
season_years=[2019,2018,2017]
seasons= {}
for year in season_years:
    seasons[year] = dict(start_YR = str(year), end_yr = str(year+1)[-2:])
seasons

{2019: {'start_YR': '2019', 'end_yr': '20'},
 2018: {'start_YR': '2018', 'end_yr': '19'},
 2017: {'start_YR': '2017', 'end_yr': '18'}}

In [38]:
fs.reload(ns)
ns.make_season_urls_from_playerid()

Reloading...



['https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2019-20&SeasonType=Regular%20Season',
 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2018-19&SeasonType=Regular%20Season',
 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2017-18&SeasonType=Regular%20Season']

In [39]:
fs.reload(ns)

df_list = ns.get_all_seasons_df(player_id=1628369)

Reloading...



In [40]:
df_list[0]

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,teams
date,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25,"Dec 25, 2019 - BOS @ TOR",W,35,11,5,18,27.8,1,3,33.3,0,0,0.0,4,5,9,6,2,0,3,4,21,BOS @ TOR
2019-12-22,"Dec 22, 2019 - BOS vs. CHA",W,36,39,15,29,51.7,4,9,44.4,5,5,100.0,1,11,12,2,1,3,2,1,10,BOS vs. CHA
2019-12-20,"Dec 20, 2019 - BOS vs. DET",W,27,26,10,17,58.8,2,7,28.6,4,4,100.0,0,5,5,3,2,0,0,1,10,BOS vs. DET
2019-12-18,"Dec 18, 2019 - BOS @ DAL",W,37,24,10,23,43.5,2,10,20.0,2,3,66.7,2,6,8,1,0,1,3,3,-2,BOS @ DAL
2019-12-12,"Dec 12, 2019 - BOS vs. PHI",L,34,15,5,18,27.8,3,8,37.5,2,2,100.0,1,2,3,4,0,1,1,4,-2,BOS vs. PHI
2019-12-11,"Dec 11, 2019 - BOS @ IND",L,37,16,4,13,30.8,2,8,25.0,6,6,100.0,0,6,6,3,2,0,3,2,-8,BOS @ IND
2019-12-09,"Dec 09, 2019 - BOS vs. CLE",W,30,19,7,14,50.0,4,6,66.7,1,2,50.0,0,11,11,2,2,2,4,1,24,BOS vs. CLE
2019-12-06,"Dec 06, 2019 - BOS vs. DEN",W,30,26,10,19,52.6,4,8,50.0,2,2,100.0,0,7,7,5,1,0,1,1,14,BOS vs. DEN
2019-12-04,"Dec 04, 2019 - BOS vs. MIA",W,34,19,6,19,31.6,2,12,16.7,5,6,83.3,1,4,5,3,1,3,3,2,31,BOS vs. MIA


In [41]:
df_combined = pd.concat(df_list,axis=0)
df_combined

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,teams
date,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25,"Dec 25, 2019 - BOS @ TOR",W,35,11,5,18,27.8,1,3,33.3,0,0,0.0,4,5,9,6,2,0,3,4,21,BOS @ TOR
2019-12-22,"Dec 22, 2019 - BOS vs. CHA",W,36,39,15,29,51.7,4,9,44.4,5,5,100.0,1,11,12,2,1,3,2,1,10,BOS vs. CHA
2019-12-20,"Dec 20, 2019 - BOS vs. DET",W,27,26,10,17,58.8,2,7,28.6,4,4,100.0,0,5,5,3,2,0,0,1,10,BOS vs. DET
2019-12-18,"Dec 18, 2019 - BOS @ DAL",W,37,24,10,23,43.5,2,10,20.0,2,3,66.7,2,6,8,1,0,1,3,3,-2,BOS @ DAL
2019-12-12,"Dec 12, 2019 - BOS vs. PHI",L,34,15,5,18,27.8,3,8,37.5,2,2,100.0,1,2,3,4,0,1,1,4,-2,BOS vs. PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-21,"Dec 21, 2017 - BOS @ NYK",L,34,17,6,10,60.0,2,6,33.3,3,4,75.0,1,2,3,0,2,0,3,1,-6,BOS @ NYK
2017-12-20,"Dec 20, 2017 - BOS vs. MIA",L,28,11,4,9,44.4,0,2,0.0,3,4,75.0,0,7,7,2,0,1,1,2,3,BOS vs. MIA
2017-12-18,"Dec 18, 2017 - BOS @ IND",W,31,16,6,8,75.0,1,1,100.0,3,3,100.0,0,5,5,4,1,1,2,3,6,BOS @ IND


In [48]:
fs.reload(ns)
df_full = ns.get_all_seasons_df(player_id=1628369)
display(df_full.head(),df_full.tail())

Reloading...



,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,teams,player_id
date,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25,"Dec 25, 2019 - BOS @ TOR",W,35,11,5,18,27.8,1,3,33.3,0,0,0.0,4,5,9,6,2,0,3,4,21,BOS @ TOR,1628369
2019-12-22,"Dec 22, 2019 - BOS vs. CHA",W,36,39,15,29,51.7,4,9,44.4,5,5,100.0,1,11,12,2,1,3,2,1,10,BOS vs. CHA,1628369
2019-12-20,"Dec 20, 2019 - BOS vs. DET",W,27,26,10,17,58.8,2,7,28.6,4,4,100.0,0,5,5,3,2,0,0,1,10,BOS vs. DET,1628369
2019-12-18,"Dec 18, 2019 - BOS @ DAL",W,37,24,10,23,43.5,2,10,20.0,2,3,66.7,2,6,8,1,0,1,3,3,-2,BOS @ DAL,1628369
2019-12-12,"Dec 12, 2019 - BOS vs. PHI",L,34,15,5,18,27.8,3,8,37.5,2,2,100.0,1,2,3,4,0,1,1,4,-2,BOS vs. PHI,1628369


,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,teams,player_id
date,,,,,,,,,,,,,,,,,,,,,,,,
2017-12-21,"Dec 21, 2017 - BOS @ NYK",L,34,17,6,10,60.0,2,6,33.3,3,4,75.0,1,2,3,0,2,0,3,1,-6,BOS @ NYK,1628369
2017-12-20,"Dec 20, 2017 - BOS vs. MIA",L,28,11,4,9,44.4,0,2,0.0,3,4,75.0,0,7,7,2,0,1,1,2,3,BOS vs. MIA,1628369
2017-12-18,"Dec 18, 2017 - BOS @ IND",W,31,16,6,8,75.0,1,1,100.0,3,3,100.0,0,5,5,4,1,1,2,3,6,BOS @ IND,1628369
2017-12-16,"Dec 16, 2017 - BOS @ MEM",W,34,19,7,13,53.8,2,2,100.0,3,3,100.0,6,4,10,1,2,2,0,1,21,BOS @ MEM,1628369
2017-12-15,"Dec 15, 2017 - BOS vs. UTA",L,26,7,1,6,16.7,1,3,33.3,4,4,100.0,0,4,4,2,0,0,1,4,-7,BOS vs. UTA,1628369


In [44]:
season_urls

['https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2019-20&SeasonType=Regular%20Season',
 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2018-19&SeasonType=Regular%20Season',
 'https://stats.nba.com/player/1628369/boxscores-traditional/?Season=2017-18&SeasonType=Regular%20Season']

In [47]:
season_urls[0].split('/')[4]

'1628369'

# SAVING COPIED MARKDOWN TABLES TO CSVS

In [66]:
# df_nba_stats_glossary = pd.read_clipboard(sep='|')

In [67]:
# df_nba_stats_glossary.columns = [col.strip() for col in df_nba_stats_glossary.columns]

In [59]:
# df_nba_stats_glossary = df_nba_stats_glossary[['COLUMN','MEANING']]

In [68]:
# df_nba_stats_glossary.drop(0,inplace=True)
# df_nba_stats_glossary

In [70]:
# df_nba_stats_glossary.to_csv('nba_stats_glossary.csv')
df_nba_stats_glossary=pd.read_csv('nba_stats_glossary.csv',index_col=0)
df_nba_stats_glossary

,COLUMN,MEANING
1,W/L,Win/Loss
2,MIN,Minutes Played
3,PTS,Points
4,FGM,Field Goals Made
5,FGA,Field Goals Attempted
6,FG%,Field Goal Percentage
7,3PM,3 Point Field Goals Made
8,3PA,3 Point Field Goals Attempted
9,3P%,3 Point Field Goals Percentage
10,FTM,Free Throws Made


In [64]:
# df_pts = pd.read_clipboard(sep='|')
# df_pts.columns = [col.strip() for col in df_pts.columns]
# df_pts.drop(0,inplace=True)
# df_pts = df_pts[['STAT','PTS']]
# df_pts

,STAT,PTS
1,3PM,1
2,AST,1.5
3,BLK,3
4,FGM,2
5,FTM,1
6,REB,1.2
7,STL,3
8,TOV,-1


In [71]:
# df_pts.to_csv('fantasy_points_conversion.csv')
df_pts = pd.read_csv('fantasy_points_conversion.csv',index_col=0)
df_pts

,STAT,PTS
1,3PM,1.0
2,AST,1.5
3,BLK,3.0
4,FGM,2.0
5,FTM,1.0
6,REB,1.2
7,STL,3.0
8,TOV,-1.0


## SAVING PLAYER CSV
- Now in ns.save_player_df

In [92]:
# import os,sys
# print(os.listdir())
# os.makedirs("data/players/" ,exist_ok=True)
# # print('\n\n',os.listdir('/data'))
df_full.head()

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,teams,player_id
date,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25,"Dec 25, 2019 - BOS @ TOR",W,35,11,5,18,27.8,1,3,33.3,0,0,0.0,4,5,9,6,2,0,3,4,21,BOS @ TOR,1628369
2019-12-22,"Dec 22, 2019 - BOS vs. CHA",W,36,39,15,29,51.7,4,9,44.4,5,5,100.0,1,11,12,2,1,3,2,1,10,BOS vs. CHA,1628369
2019-12-20,"Dec 20, 2019 - BOS vs. DET",W,27,26,10,17,58.8,2,7,28.6,4,4,100.0,0,5,5,3,2,0,0,1,10,BOS vs. DET,1628369
2019-12-18,"Dec 18, 2019 - BOS @ DAL",W,37,24,10,23,43.5,2,10,20.0,2,3,66.7,2,6,8,1,0,1,3,3,-2,BOS @ DAL,1628369
2019-12-12,"Dec 12, 2019 - BOS vs. PHI",L,34,15,5,18,27.8,3,8,37.5,2,2,100.0,1,2,3,4,0,1,1,4,-2,BOS vs. PHI,1628369


In [91]:
filename=f"data/players/player_id_{df_full['player_id'][0]}.csv"
filename
df_full.to_csv(filename)

# CALCULATING FANTASY POINTS

In [93]:
points = pd.read_csv('fantasy_points_conversion.csv',index_col=0)
points

,STAT,PTS
1,3PM,1.0
2,AST,1.5
3,BLK,3.0
4,FGM,2.0
5,FTM,1.0
6,REB,1.2
7,STL,3.0
8,TOV,-1.0


In [111]:
pts = dict(zip(points['STAT'],points['PTS']))
pts

{'3PM': 1.0,
 'AST': 1.5,
 'BLK': 3.0,
 'FGM': 2.0,
 'FTM': 1.0,
 'REB': 1.2,
 'STL': 3.0,
 'TOV': -1.0}

In [114]:
df_fantasy = df_full.copy()
pts_cols= []
for k,v in pts.items():
    col_name = k+'_pts'
    df_fantasy[col_name] = df_fantasy[k].apply(lambda x: x*pts[k])
    pts_cols.append(col_name)


df_fantasy['total_pts'] = df_fantasy.apply(lambda x: x[pts_cols].sum(),axis=1)
df_fantasy
    

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,teams,player_id,3PM_pts,AST_pts,BLK_pts,FGM_pts,FTM_pts,REB_pts,STL_pts,TOV_pts,total_pts
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25,"Dec 25, 2019 - BOS @ TOR",W,35,11,5,18,27.8,1,3,33.3,0,0,0.0,4,5,9,6,2,0,3,4,21,BOS @ TOR,1628369,1.0,9.0,0.0,10.0,0.0,10.8,6.0,-3.0,33.8
2019-12-22,"Dec 22, 2019 - BOS vs. CHA",W,36,39,15,29,51.7,4,9,44.4,5,5,100.0,1,11,12,2,1,3,2,1,10,BOS vs. CHA,1628369,4.0,3.0,9.0,30.0,5.0,14.4,3.0,-2.0,66.4
2019-12-20,"Dec 20, 2019 - BOS vs. DET",W,27,26,10,17,58.8,2,7,28.6,4,4,100.0,0,5,5,3,2,0,0,1,10,BOS vs. DET,1628369,2.0,4.5,0.0,20.0,4.0,6.0,6.0,-0.0,42.5
2019-12-18,"Dec 18, 2019 - BOS @ DAL",W,37,24,10,23,43.5,2,10,20.0,2,3,66.7,2,6,8,1,0,1,3,3,-2,BOS @ DAL,1628369,2.0,1.5,3.0,20.0,2.0,9.6,0.0,-3.0,35.1
2019-12-12,"Dec 12, 2019 - BOS vs. PHI",L,34,15,5,18,27.8,3,8,37.5,2,2,100.0,1,2,3,4,0,1,1,4,-2,BOS vs. PHI,1628369,3.0,6.0,3.0,10.0,2.0,3.6,0.0,-1.0,26.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-21,"Dec 21, 2017 - BOS @ NYK",L,34,17,6,10,60.0,2,6,33.3,3,4,75.0,1,2,3,0,2,0,3,1,-6,BOS @ NYK,1628369,2.0,0.0,0.0,12.0,3.0,3.6,6.0,-3.0,23.6
2017-12-20,"Dec 20, 2017 - BOS vs. MIA",L,28,11,4,9,44.4,0,2,0.0,3,4,75.0,0,7,7,2,0,1,1,2,3,BOS vs. MIA,1628369,0.0,3.0,3.0,8.0,3.0,8.4,0.0,-1.0,24.4
2017-12-18,"Dec 18, 2017 - BOS @ IND",W,31,16,6,8,75.0,1,1,100.0,3,3,100.0,0,5,5,4,1,1,2,3,6,BOS @ IND,1628369,1.0,6.0,3.0,12.0,3.0,6.0,3.0,-2.0,32.0


In [115]:
fs.reload(ns)
test_df = ns.calc_fantasy_stats(df_full)

Reloading...



In [116]:
test_df

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,teams,player_id,3PM_pts,AST_pts,BLK_pts,FGM_pts,FTM_pts,REB_pts,STL_pts,TOV_pts,total_pts
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-25,"Dec 25, 2019 - BOS @ TOR",W,35,11,5,18,27.8,1,3,33.3,0,0,0.0,4,5,9,6,2,0,3,4,21,BOS @ TOR,1628369,1.0,9.0,0.0,10.0,0.0,10.8,6.0,-3.0,33.8
2019-12-22,"Dec 22, 2019 - BOS vs. CHA",W,36,39,15,29,51.7,4,9,44.4,5,5,100.0,1,11,12,2,1,3,2,1,10,BOS vs. CHA,1628369,4.0,3.0,9.0,30.0,5.0,14.4,3.0,-2.0,66.4
2019-12-20,"Dec 20, 2019 - BOS vs. DET",W,27,26,10,17,58.8,2,7,28.6,4,4,100.0,0,5,5,3,2,0,0,1,10,BOS vs. DET,1628369,2.0,4.5,0.0,20.0,4.0,6.0,6.0,-0.0,42.5
2019-12-18,"Dec 18, 2019 - BOS @ DAL",W,37,24,10,23,43.5,2,10,20.0,2,3,66.7,2,6,8,1,0,1,3,3,-2,BOS @ DAL,1628369,2.0,1.5,3.0,20.0,2.0,9.6,0.0,-3.0,35.1
2019-12-12,"Dec 12, 2019 - BOS vs. PHI",L,34,15,5,18,27.8,3,8,37.5,2,2,100.0,1,2,3,4,0,1,1,4,-2,BOS vs. PHI,1628369,3.0,6.0,3.0,10.0,2.0,3.6,0.0,-1.0,26.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-21,"Dec 21, 2017 - BOS @ NYK",L,34,17,6,10,60.0,2,6,33.3,3,4,75.0,1,2,3,0,2,0,3,1,-6,BOS @ NYK,1628369,2.0,0.0,0.0,12.0,3.0,3.6,6.0,-3.0,23.6
2017-12-20,"Dec 20, 2017 - BOS vs. MIA",L,28,11,4,9,44.4,0,2,0.0,3,4,75.0,0,7,7,2,0,1,1,2,3,BOS vs. MIA,1628369,0.0,3.0,3.0,8.0,3.0,8.4,0.0,-1.0,24.4
2017-12-18,"Dec 18, 2017 - BOS @ IND",W,31,16,6,8,75.0,1,1,100.0,3,3,100.0,0,5,5,4,1,1,2,3,6,BOS @ IND,1628369,1.0,6.0,3.0,12.0,3.0,6.0,3.0,-2.0,32.0


# WORKFLOW SUMMARY

1. Get player's nba_stats id#
2. `import nba_stats as ns`
3. Can directly run `df = ns.get_all_seasons_df(player_id= the_players_id)` (see note below: \$:)
4. `df = ns.calc_fantasy_stats(df)`
5. `ns.save_player_df(df)`

\$: Can instead run helper functions individually if don't want all seasons
- `season_urls=ns.make_season_urls_from_playerid`
- `df = ns.get_season_df(season_url[0])`





In [118]:
import nba_stats as ns
player_id=1628369
df = ns.get_all_seasons_df(player_id=player_id)
df = ns.calc_fantasy_stats(df)
ns.save_player_df(df)


In [0]:
df